<a href="https://colab.research.google.com/github/RetoHe/epex-spot-intraday-timeseries-analysis/blob/main/Time_Series_Forecasting_EPEX_Intraday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Time Series Forecasting with EPEX Spot Intraday Data
### Basic Feature Engineering for Time Series inspired from *Introduction to Time Series Forecasting with Python* from Jason Brownlee.

## Installieren der Requirements

In [3]:
%%capture
! pip install pysftp

In [4]:
%%capture
!pip install statsmodels

In [5]:
import pysftp
import pandas as pd

## Verbindung zu SFTP Server

In [6]:
%%capture
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

/usr/local/lib/python3.7/dist-packages/pysftp/__init__.py:61: UserWarning: Failed to load HostKeys from /root/.ssh/known_hosts.  You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).
  warnings.warn(wmsg, UserWarning)


In [9]:
sftp = pysftp.Connection(host="XXXXX", username="XXXXX", password="XXXXX", cnopts=cnopts)

In [10]:
sftp.chdir("/germany/Intraday Continuous/EOD/Transactions")

In [11]:
!mkdir data

## Download der Daten

In [84]:
german_dir = "/germany/Intraday Continuous/EOD/Transactions/"
# just export one single day of 2021
for element in sftp.listdir():
    if element[-3:] == "csv":
        new_path = german_dir + element
        sftp.get(new_path, "data/intradaydata_DE_{}.csv".format(element[29:40]))
        break
    else:
        pass
    

OSError: ignored

### Beispiel Daten Intraday Trades German Market 01.01.2021

In [18]:
data = pd.read_csv("data/intradaydata_DE__2021-01-01.csv", dtype={'Date': 'str', 'Market Area Buy': 'str', 'Market Area Sell': 'str', 'Hour from': 'str', 'Hour to': 'str', 'Quantity (MW)': float, 'Price (EUR)': float, 'Time Stamp': 'str', 'Trade ID': 'str'}, skiprows=[0], parse_dates=["Date", "Time Stamp"])

In [20]:
# Choose data for Product Hour 8
data = data[data["Hour from"]=="8"]

In [24]:
# Remove unnecessary columns
# Date, Price and Time Stamp are needed
data = data[['Date', 'Price (EUR)', "Time Stamp"]]

In [29]:
# Rename Price column
data = data.rename(columns={'Price (EUR)': 'Price'})

In [30]:
data.head()

,Date,Price,Time Stamp
192,2021-01-01,31.36,2020-12-31 17:02:16
335,2021-01-01,39.65,2020-12-31 17:20:54
336,2021-01-01,42.50,2020-12-31 17:21:12
381,2021-01-01,42.00,2020-12-31 17:28:46
686,2021-01-01,39.45,2020-12-31 18:01:16


## Lag Features Time Series Data

In [44]:
# Create Input Feature for T+1
dataframe = pd.concat([data["Price"].shift(1), data["Price"]], axis = 1)
dataframe.columns = ["t", "t+1"]

In [45]:
dataframe.head()

,t,t+1
192,NaN,31.36
335,31.36,39.65
336,39.65,42.50
381,42.50,42.00
686,42.00,39.45


In [46]:
# Create more Input Features with Lag
dataframe = pd.concat([data["Price"].shift(3), data["Price"].shift(2), data["Price"].shift(1), data["Price"]], axis = 1)
dataframe.columns = ["t-2", "t-1", "t", "t+1"]

In [47]:
dataframe.head()

,t-2,t-1,t,t+1
192,NaN,NaN,NaN,31.36
335,NaN,NaN,31.36,39.65
336,NaN,31.36,39.65,42.50
381,31.36,39.65,42.50,42.00
686,39.65,42.50,42.00,39.45


## Rolling Window Statistics

In [49]:
# Create Mean of last 2 time steps as new input feature
prices = data["Price"]
shifted = prices.shift(1)
window = shifted.rolling(window=2)
means = window.mean()

In [52]:
dataframe = pd.concat([means, prices], axis = 1)
dataframe.columns = ["means(t-1, t)", "t+1"]

In [53]:
dataframe.head()

,"means(t-1, t)",t+1
192,NaN,31.36
335,NaN,39.65
336,35.505,42.50
381,41.075,42.00
686,42.250,39.45


In [54]:
# Use Min, Mean and Max Value of last 3 time steps as input feature
prices = data["Price"]
width = 3
shifted = prices.shift(width - 1)
window = shifted.rolling(window=width)



In [58]:
dataframe = pd.concat([window.min(), window.mean(), window.max(), prices], axis = 1)
dataframe.columns = ["min", "mean", "max", "t+1"]

In [59]:
dataframe.head()

,min,mean,max,t+1
192,NaN,NaN,NaN,31.36
335,NaN,NaN,NaN,39.65
336,NaN,NaN,NaN,42.50
381,NaN,NaN,NaN,42.00
686,31.36,37.836667,42.5,39.45


## Expanding Window Statistics

In [65]:
prices = data["Price"]
window = prices.expanding()
dataframe = pd.concat([window.min(), window.mean(), window.max(), prices.shift(-1)], axis = 1)
dataframe.columns = ["min", "mean", "max", "t+1"]

In [66]:
dataframe.head()

,min,mean,max,t+1
192,31.36,31.360000,31.36,39.65
335,31.36,35.505000,39.65,42.50
336,31.36,37.836667,42.50,42.00
381,31.36,38.877500,42.50,39.45
686,31.36,38.992000,42.50,39.45


# Moving Average Smoothing

In [71]:
# Use Moving Average as Input Feature
prices = data["Price"]
width = 3
lag1 = prices.shift(1)
lag3 = prices.shift(width-1)
window = lag3.rolling(window=width)
means = window.mean()
dataframe = pd.concat([means, lag1, prices], axis = 1)
dataframe.columns = ["mean", "t", "t+1"]

In [72]:
dataframe.head()

,mean,t,t+1
192,NaN,NaN,31.36
335,NaN,31.36,39.65
336,NaN,39.65,42.50
381,NaN,42.50,42.00
686,37.836667,42.00,39.45


## Moving Average as Prediction

In [81]:
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from numpy import mean
from math import sqrt

In [82]:
# Use Moving Average Method as Time Series Prediction Method
X = data["Price"].values
window = 3
history = [X[i] for i in range(window)]
test = [X[i] for i in range(window, len(X))]
predictions = []

# walk forward over time steps in test

for t in range(len(test)):
  length = len(history)
  yhat = mean([history[i] for i in range(length-window, length)])
  obs = test[t]

  predictions.append(yhat)
  history.append(obs)

  print("predicted= %f, expected= %f" % (yhat, obs))


rmse = sqrt(mean_squared_error(test, predictions))

print("Test RMSE: %.3f" % rmse)

predicted= 37.836667, expected= 42.000000
predicted= 41.383333, expected= 39.450000
predicted= 41.316667, expected= 39.450000
predicted= 40.300000, expected= 39.450000
predicted= 39.450000, expected= 39.500000
predicted= 39.466667, expected= 40.750000
predicted= 39.900000, expected= 40.300000
predicted= 40.183333, expected= 39.900000
predicted= 40.316667, expected= 39.400000
predicted= 39.866667, expected= 39.990000
predicted= 39.763333, expected= 39.870000
predicted= 39.753333, expected= 39.750000
predicted= 39.870000, expected= 39.000000
predicted= 39.540000, expected= 39.000000
predicted= 39.250000, expected= 39.990000
predicted= 39.330000, expected= 40.440000
predicted= 39.810000, expected= 40.660000
predicted= 40.363333, expected= 40.660000
predicted= 40.586667, expected= 39.700000
predicted= 40.340000, expected= 39.650000
predicted= 40.003333, expected= 39.650000
predicted= 39.666667, expected= 39.650000
predicted= 39.650000, expected= 39.650000
predicted= 39.650000, expected= 40